In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

In [3]:
def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [4]:
def move_forward(vehicle):
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)

def stop_vehicle(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.0
    control.brake = 1.0
    vehicle.apply_control(control)
 

In [5]:
from enum import Enum

class Fcw_state(Enum):
    IDLE = 4
    WARNING = 3
    ACTION = 2
    ESCAPE = 1

In [6]:
def check_lateral_collision(azimuth, depth):
    global vehicle_lateral_dimension
    return abs(depth * math.sin(azimuth)) < vehicle_lateral_dimension


def check_vertical_collision(altitude, depth):
    global vehicle_vertical_dimension
    return abs(depth * math.sin(altitude)) < vehicle_vertical_dimension 

def get_projected_velocity(azimuth, altitude, velocity):
    return abs(math.cos(azimuth) * math.cos(altitude) * velocity)

def get_projected_depth(azimuth, altitude, depth):
    return abs(math.cos(azimuth) * math.cos(altitude) * depth)

def get_breaking_distance(projected_velocity):
    global asphalt_friction_deceleration
    return (1/2) * projected_velocity**2 / asphalt_friction_deceleration

def get_vehicle_velocity(vehicle):
    velocity_vector = vehicle.get_velocity()
    velocity = (velocity_vector.x**2 + velocity_vector.y**2 + velocity_vector.z**2)**0.5
    if vehicle.get_control().reverse:
        velocity = -velocity
    return velocity

In [8]:
counter = 0

def forward_collision_callback(radar_data, vehicle):
        global min_fcw_state, counter, min_ttc, velocity_threshold, average_reaction_time, free_from_obstacle_counter
        obastacle_detected = False
        for detection in radar_data:
            if detection.velocity < -velocity_threshold and check_lateral_collision(detection.azimuth, detection.depth) and check_vertical_collision(detection.altitude, detection.depth):
                  projected_velocity = get_projected_velocity(detection.azimuth, detection.altitude, detection.velocity)
                  if projected_velocity > velocity_threshold:
                    breaking_distance = (projected_velocity)
                    projected_depth = get_projected_depth(detection.azimuth, detection.altitude, detection.depth)
                    reacting_distance = max(0, projected_depth - breaking_distance)
                    ttc = reacting_distance / projected_velocity
                    obastacle_detected = True
                    fcw_state = Fcw_state.IDLE
                    if ttc < min_ttc:
                      vehicle_velocity = get_vehicle_velocity(vehicle)
                      if vehicle_velocity < projected_velocity - 2*velocity_threshold:
                        fcw_state = Fcw_state.ESCAPE
                      else:
                        fcw_state = Fcw_state.ACTION
                    elif ttc < min_ttc + average_reaction_time + 1:
                         fcw_state = Fcw_state.WARNING
                    if fcw_state.value < min_fcw_state.value:
                        min_fcw_state = fcw_state

        if not obastacle_detected:
            counter += 1
            if counter > free_from_obstacle_counter:
              counter = 0
              min_fcw_state = Fcw_state.IDLE   
        else: 
            counter = 0      

In [14]:
def create_radar(vehicle):
    rad_bp = world.get_blueprint_library().find('sensor.other.radar')
    rad_bp.set_attribute('horizontal_fov', str(125))
    rad_bp.set_attribute('vertical_fov', str(115))
    rad_bp.set_attribute('range', str(285))
    rad_bp.set_attribute('points_per_second', str(2000))
    rad_location = carla.Location(x=2.25, z=0.9)
    rad_rotation = carla.Rotation()
    rad_transform = carla.Transform(rad_location,rad_rotation)
    return world.spawn_actor(rad_bp,rad_transform,attach_to=vehicle, attachment_type=carla.AttachmentType.Rigid)


In [10]:
def get_asphalt_friction_coefficient():
    world = client.get_world()
    weather = world.get_weather()
    weather.precipitation
    weather.precipitation_deposits
    weather.wetness
    weather.dust_storm
    return 0.8

In [7]:
walker_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))
controller_bp = world.get_blueprint_library().find('controller.ai.walker')

start_location = carla.Location(x=70, y=25, z=1)

trans = carla.Transform()
trans.location = start_location

walker = world.spawn_actor(walker_bp, trans)

In [17]:
min_fcw_state = Fcw_state.IDLE
vehicle_vertical_dimension = 0.75
vehicle_lateral_dimension = 0.9
min_ttc = 0.5
average_reaction_time = 2.5
throttle = 0.5
free_from_obstacle_counter = 10
velocity_threshold = 2.77
asphalt_friction_coefficient = 0.8
asphalt_friction_deceleration = 9.81 * asphalt_friction_coefficient

try:
    vehicle = spawn_vehicle()
    radar = create_radar(vehicle)
    #vehicle.set_autopilot(True)
    radar.listen(lambda radar_data: forward_collision_callback(radar_data, vehicle))
    while True:
        if min_fcw_state == Fcw_state.ACTION:
            stop_vehicle(vehicle)
            print("Emergency braking activated!")  
            time.sleep(10)
        elif min_fcw_state == Fcw_state.WARNING:    
            print("Emergency warning!")  
        elif min_fcw_state == Fcw_state.ESCAPE: 
            print("EscapeTime!") 
        else: 
            move_forward(vehicle)
        asphalt_friction_coefficient = get_asphalt_friction_coefficient()
        asphalt_friction_deceleration = 9.81 * asphalt_friction_coefficient
        print(get_vehicle_velocity(vehicle))
        time.sleep(0.05)

finally:
    vehicle.destroy()
    radar.destroy()
    

1.0603449336865207
0.2671478730563476
0.2743607825481742
0.168295029658721
0.07674196243991482
0.024650255525097364
0.002512284018676644
0.0038099991380047622
0.0
0.002091855157268381
0.0016720893713833517
0.9072150502896454
1.1989235510687946
1.2353860399009609
1.2970339528663561
1.372200594573142
1.4527352359409214
1.5338143002523914
1.6128431923547286
1.688523480846645
1.7602858214736883
1.8279527128373263
1.8915595597456853
1.9512488171472553
2.0072137248341373
2.0596669542937547
2.108824750069103
2.1548976364714663
2.1980858868582405
2.2385761844452454
2.2765442422009383
2.3121560009272026
2.3455635718601067
2.376910007466698
2.406327629411676
2.433938986400998
2.4599538852370992
2.4848882691369907
2.509073538607256
2.5326591832104093
2.555717683006696
2.578284085195432
2.6003786546642655
2.622015936226816
2.6432092805648524
2.6639675102110822
2.684300398563385
2.7042193914428205
2.7237328330098367
2.7428195033783895
2.7614505550757196
2.7796753399911855
2.797524852594525
2.815013

KeyboardInterrupt: 

In [ ]:
walker.destroy()